In [361]:
import numpy as np
import pandas as pd
from random import random
from math import exp
from random import seed

In [362]:
data = pd.read_csv('data/seeds_dataset.txt',delim_whitespace=True,header=None)
features = list(data)[:-1]
target = 7
for feature in features:
    curr_max = max(data[feature])
    curr_min = min(data[feature])
    data[feature] = data[feature].apply(lambda x : x/(curr_max-curr_min))
data['output'] = data[target].apply(lambda x: [1 if i==x-1 else 0 for i in range(3)])
data[target] = data[target].astype(int)

In [363]:
data.dtypes

0         float64
1         float64
2         float64
3         float64
4         float64
5         float64
6         float64
7           int64
output     object
dtype: object

# Initialize the network

In [364]:
def initialize_network(n_inputs, n_hidden,n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [365]:
#test network
network = initialize_network(2,1,2)
for layer in network:
    print layer

[{'weights': [0.4896935204622582, 0.029574963966907064, 0.04348729035652743]}]
[{'weights': [0.703382088603836, 0.9831877173096739]}, {'weights': [0.5931837303800576, 0.393599686377914]}]


# Activation of neurons

In [366]:
def activate(weights,inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

# Transfer neuron activation

In [367]:
def transfer(activation):
    return 1.0/(1.0 + exp(-activation))

# Forward propagation

In [368]:
def forward_propagate(network,row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'],inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [369]:
# Testing forward propagation
network = initialize_network(2,1,2)
row = [1,0,None]     # Last one is the dummy input(bias 1)
output = forward_propagate(network,row)
print output

[0.7549481739306217, 0.7080550424834332]


# Back propagation

In [370]:
def transfer_derivative(output):
    return output*(1-output)

In [371]:
def back_propagate_error(network,expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*transfer_derivative(neuron['output'])

In [372]:
# Testing backpropagation
expected = [0,1]
back_propagate_error(network,expected)

In [373]:
for layer in network:
    print layer

[{'output': 0.7599537265289951, 'weights': [0.17034919685568128, 0.5022385584334831, 0.9820766375385342], 'delta': -0.01016079516351957}]
[{'output': 0.7549481739306217, 'weights': [0.7705231398308006, 0.5396174484497788], 'delta': -0.1396664907032541}, {'output': 0.7080550424834332, 'weights': [0.8602897789205496, 0.23217612806301458], 'delta': 0.06034884699244381}]


# Updating network weights after getting errors

In [374]:
def update_weights(network,row,l_rate): #l_rate:learining rate
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] += l_rate*neuron['delta']

# Training network

In [375]:
def train_network(network,train,l_rate,n_epoch,n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network,row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]-1] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            back_propagate_error(network,expected)
            update_weights(network,row,l_rate)
        print "Iteration : ",epoch,", Learning rate : ",l_rate,", Sum of errors : ",sum_error
    

In [376]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0])-1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs,2,n_outputs)
train_network(network,dataset,0.5,20,n_outputs)


Iteration :  0 , Learning rate :  0.5 , Sum of errors :  6.36560716772
Iteration :  1 , Learning rate :  0.5 , Sum of errors :  5.59766450753
Iteration :  2 , Learning rate :  0.5 , Sum of errors :  5.37370225047
Iteration :  3 , Learning rate :  0.5 , Sum of errors :  5.32681688461
Iteration :  4 , Learning rate :  0.5 , Sum of errors :  5.26669974755
Iteration :  5 , Learning rate :  0.5 , Sum of errors :  5.13742274206
Iteration :  6 , Learning rate :  0.5 , Sum of errors :  4.96460675094
Iteration :  7 , Learning rate :  0.5 , Sum of errors :  4.77357774934
Iteration :  8 , Learning rate :  0.5 , Sum of errors :  4.54896132634
Iteration :  9 , Learning rate :  0.5 , Sum of errors :  4.28425562944
Iteration :  10 , Learning rate :  0.5 , Sum of errors :  3.98903023251
Iteration :  11 , Learning rate :  0.5 , Sum of errors :  3.67880171937
Iteration :  12 , Learning rate :  0.5 , Sum of errors :  3.36848702887
Iteration :  13 , Learning rate :  0.5 , Sum of errors :  3.06977488333
It

In [377]:
for layer in network:
    print layer
    print ""

[{'output': 0.9965606621574864, 'weights': [0.38645713364013506, 0.5764659087897724, 0.6916964513810357], 'delta': 0.0002994684914070449}, {'output': 0.03382652687004891, 'weights': [-1.3191491497141457, 1.6205210721287375, 0.811885442248185], 'delta': -0.0075667186028044895}]

[{'output': 0.7065695133371781, 'weights': [0.8258105235746557, -1.900248018719592, 0.18075191605186908], 'delta': 0.06083665997965738}, {'output': 0.256453313500061, 'weights': [-0.8214066848336647, 2.3683320203807443, -0.3747776593797591], 'delta': -0.048901803032664434}]



# prediction

In [378]:
def predict(network,row):
    outputs = forward_propagate(network,row)
    return outputs.index(max(outputs))

In [379]:
for row in dataset:
    prediction = predict(network,row)
    print "Expected : ",row[-1], ", Got : ",prediction

Expected :  0 , Got :  1
Expected :  0 , Got :  1
Expected :  0 , Got :  1
Expected :  0 , Got :  1
Expected :  0 , Got :  1
Expected :  1 , Got :  0
Expected :  1 , Got :  0
Expected :  1 , Got :  0
Expected :  1 , Got :  0
Expected :  1 , Got :  0


# Binding things together

In [380]:
def back_propagation(train,l_rate,n_epoch,n_hidden):
    n_inputs = len(train[0])-1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs,n_hidden,n_outputs)
    train_network(network,train,l_rate,n_epoch,n_outputs)
    return network


In [381]:
train = []
for i in range(len(data)):
    curr = list(data[features].iloc[i])
    curr.append(int(data[target].iloc[i]))
    train.append(list(curr))

In [385]:
my_network = back_propagation(train,0.3,500,5)
my_network

Iteration :  0 , Learning rate :  0.3 , Sum of errors :  98.0153413192
Iteration :  1 , Learning rate :  0.3 , Sum of errors :  70.6703123588
Iteration :  2 , Learning rate :  0.3 , Sum of errors :  70.9005068008
Iteration :  3 , Learning rate :  0.3 , Sum of errors :  70.9015366379
Iteration :  4 , Learning rate :  0.3 , Sum of errors :  70.9013425253
Iteration :  5 , Learning rate :  0.3 , Sum of errors :  70.9011483797
Iteration :  6 , Learning rate :  0.3 , Sum of errors :  70.9009604549
Iteration :  7 , Learning rate :  0.3 , Sum of errors :  70.9007784843
Iteration :  8 , Learning rate :  0.3 , Sum of errors :  70.9006021852
Iteration :  9 , Learning rate :  0.3 , Sum of errors :  70.9004312927
Iteration :  10 , Learning rate :  0.3 , Sum of errors :  70.9002655581
Iteration :  11 , Learning rate :  0.3 , Sum of errors :  70.9001047477
Iteration :  12 , Learning rate :  0.3 , Sum of errors :  70.899948642
Iteration :  13 , Learning rate :  0.3 , Sum of errors :  70.8997970343
Ite

[[{'delta': 2.3556998487622365e-06,
   'output': 0.9995998989460628,
   'weights': [0.49470682751665829,
    0.056952087244238843,
    0.14195672678604093,
    0.73559258725402832,
    0.99914194248607324,
    0.60304830296582668,
    0.42422493622194418,
    0.18421745998064967]},
  {'delta': 1.1268857977635933e-09,
   'output': 0.9999997445149991,
   'weights': [0.50221597101626769,
    0.98201612130863958,
    0.77036146601328259,
    0.53955003393060963,
    0.86024933885944033,
    0.23215631864753802,
    0.51370974518478008,
    0.9524449058575325]},
  {'delta': 2.181916256656266e-07,
   'output': 0.9999557034847774,
   'weights': [0.57225909277267628,
    0.44511991683859065,
    0.23026081496819506,
    0.53277840208459581,
    0.94699555629654442,
    0.0024675947190513571,
    0.77068107815523512,
    0.8153256110373067]},
  {'delta': 6.2502250413812606e-09,
   'output': 0.9999985871180664,
   'weights': [0.88594603049936749,
    0.73989357246967535,
    0.80745328349389145,

In [386]:

output = []
for i in range(len(train)):
    curr = forward_propagate(my_network,train[i])
    output.append(curr)

In [388]:
for x in output:
    print x

[0.05106627334147904, 0.07894221970075141, 0.9166843197949434]
[0.05106756201504071, 0.07894315395681527, 0.9166803590072108]
[0.05106771248034762, 0.07894321886172918, 0.9166803001329544]
[0.05106796190995481, 0.0789434617202166, 0.9166796076312308]
[0.05106544808728486, 0.07894154451376481, 0.9166865054101627]
[0.05106753340864485, 0.07894310300288035, 0.916680737750695]
[0.051066536015494574, 0.07894239572257034, 0.9166837930867008]
[0.051067478534945594, 0.07894308317978055, 0.9166809957702542]
[0.05106431633839258, 0.0789407217623846, 0.9166898496456236]
[0.0510647398789298, 0.07894103204089017, 0.9166886334904261]
[0.05106575534887577, 0.0789418366440379, 0.9166861763054838]
[0.05106850860556654, 0.07894384171034965, 0.9166777620212335]
[0.051067919586190655, 0.07894343656002478, 0.9166802246684218]
[0.05106834803758221, 0.0789437685679392, 0.9166787473600115]
[0.051068776490829804, 0.07894408011097666, 0.9166774883593701]
[0.05106699233884533, 0.0789427250452382, 0.9166827420723